In [1]:
import pandas as pd
import ast
from tqdm import tqdm
tqdm.pandas()

In [2]:
# Load Data
df_yake_claims = pd.read_json('/mnt/hdd01/patentsview/Patentsview - Cleantech Patents/Cleantech Concepts/Yake/g_patent_claims_cleantech_yake.json')

In [3]:
# Iterate over rows in dataframe
for index, row in tqdm(df_yake_claims.iterrows()):
    # Cast row 'keywords_yake' column to string
    row['keywords_yake'] = str(row['keywords_yake']).lower()
    # Check if keywords_yake column starts with "[[[" and ends with "]]]""
    if row['keywords_yake'].startswith('[[[') and row['keywords_yake'].endswith(']]]'):
        # Remove first "[" and last "]" from keywords_yake column
        row['keywords_yake'] = row['keywords_yake'][1:-1]
    # Cast row 'keywords_yake' column to list
    row['keywords_yake'] = ast.literal_eval(row['keywords_yake'])
    # Assign modified 'keywords_yake' list to temporary variable
    keywords_yake_temp = row['keywords_yake']
    # Assign temporary variable to 'keywords_yake' column
    df_yake_claims.at[index, 'keywords_yake'] = keywords_yake_temp

515742it [01:11, 7163.72it/s]


In [4]:
df_yake_claims['keywords_yake_exploded'] = ''
# Iterate over rows in dataframe
for index,row in tqdm(df_yake_claims.iterrows()):
    # Check if 'keywords_yake' column is not a list
    if not isinstance(row['keywords_yake'], list):
        continue
    # Check if 'keywords_yake' column is an empty list or contains only empty lists
    if not any(keyword for keyword in row['keywords_yake']):
        continue
    # Iterate over keywords in 'keywords_yake' column and append to 'keywords_yake_exploded' column
    else:
        for keyword in row['keywords_yake']:
            df_yake_claims.at[index, 'keywords_yake_exploded'] += keyword[0] + ', '

515742it [02:06, 4070.62it/s]


In [5]:
# Cast 'keywords_yake_exploded' column to list
df_yake_claims['keywords_yake_exploded'] = df_yake_claims['keywords_yake_exploded'].str.split(', ')

In [6]:
# Delete last item in 'keywords_yake_exploded' cells if empty string
df_yake_claims['keywords_yake_exploded'] = df_yake_claims['keywords_yake_exploded'].progress_apply(lambda x: x[:-1] if x and x[-1] == '' else x)

100%|██████████| 515742/515742 [00:00<00:00, 1495513.99it/s]


In [7]:
keywords_list = []
min_yake_conf = 0.05

# Iterate over rows in dataframe
for index,row in tqdm(df_yake_claims.iterrows()):
    # Check if 'keywords_yake' column is not a list
    if not isinstance(row['keywords_yake'], list):
        continue
    # Check if 'keywords_yake' column is an empty list or contains only empty lists
    if not any(keyword for keyword in row['keywords_yake']):
        continue
    # Iterate over keywords in 'keywords_yake' column and append to 'keywords_yake_exploded' column
    else:
        for keyword in row['keywords_yake']:
            if keyword[1] <= min_yake_conf:
                keywords_list.append(keyword[0])

515742it [00:16, 31662.42it/s]


In [9]:
# Only consider alphanumeric characters in keywords
keywords_list = [
    keyword for keyword in keywords_list 
    if all(word.isalnum() for word in keyword.split())
]

# Postprocessing

In [10]:
# Generate unique list of keywords
keywords_list_unique = list(set(keywords_list))
df_keywords_list_unique = pd.DataFrame(keywords_list_unique, columns=['keyword_yake'])

In [12]:
# Count absolute frequency of each keyword
from collections import Counter

# Step 1: Flatten the list of keywords into a single list
flattened_keywords = [keyword for keyword_list in tqdm(df_yake_claims['keywords_yake_exploded']) for keyword in keyword_list]

# Step 2: Use collections.Counter to count the occurrences of each keyword
keyword_counts = Counter(flattened_keywords)

# Step 3: Convert the keyword counts to a DataFrame
count_df = pd.DataFrame(keyword_counts.items(), columns=['keyword_yake', 'abs_frequency'])

# Step 4: Merge the count_df with df_keywords_list_unique on the 'keyword_yake' column
df_keywords_list_unique = df_keywords_list_unique.merge(count_df, on='keyword_yake', how='left')

100%|██████████| 515742/515742 [00:00<00:00, 2761819.73it/s]


In [ ]:
### MAYBE I NEED TO REDO THIS WITH FULL TEXT CLAIMS INSTEAD OF JUST THE YAKE KEYWORDS

# Calculate document frequency by dividing absolute frequency by len of df_yake_claims
# df_keywords_list_unique['doc_frequency'] = df_keywords_list_unique['abs_frequency'] / len(df_yake_claims)

In [13]:
# Prune dataframe by document frequency and absolute frequency
min_abs_frequency = 5
max_abs_frequency = 1000
# max_doc_frequency = 0.3

df_keywords_list_unique_pruned = df_keywords_list_unique[(df_keywords_list_unique['abs_frequency'] >= min_abs_frequency) & (df_keywords_list_unique['abs_frequency'] <= max_abs_frequency)]

# Reset index
df_keywords_list_unique_pruned.reset_index(drop=True, inplace=True)

In [ ]:
# Only consider n-grams with n >= 2 - NOT CONSIDERED FOR NOW
# min_ngram_length = 2

# # Generate copy of df_keywords_list_unique_pruned
# df_keywords_list_unique_pruned = df_keywords_list_unique_pruned.copy()

# def filter_keywords(keyword_string):
#     # Split the keyword string into individual keywords
#     keywords = keyword_string.split(', ')
#     # Filter out keywords with fewer than 2 words
#     filtered_keywords = [keyword for keyword in keywords if len(keyword.split()) >= min_ngram_length]
#     # Join the filtered keywords back into a string
#     return ', '.join(filtered_keywords)

# # Create a copy of df_keywords_list_unique_pruned to avoid SettingWithCopyWarning
# df_keywords_list_unique_pruned = df_keywords_list_unique_pruned.copy()

# # Now use .loc to modify the 'keyword_yake' column
# df_keywords_list_unique_pruned.loc[:, 'keyword_yake'] = df_keywords_list_unique_pruned['keyword_yake'].progress_apply(filter_keywords)

# # Delete rows with empty 'keyword_yake' column
# df_keywords_list_unique_pruned = df_keywords_list_unique_pruned[df_keywords_list_unique_pruned['keyword_yake'] != '']

# # Reset index
# df_keywords_list_unique_pruned.reset_index(drop=True, inplace=True)

In [14]:
import nltk
from nltk.corpus import stopwords

# Download stopwords
# nltk.download('stopwords')

# Generate copy of df_keywords_list_unique_pruned
df_keywords_list_unique_pruned = df_keywords_list_unique_pruned.copy()

# Delete all stopwords from 'keyword_yake' column
df_keywords_list_unique_pruned.loc[:, 'keyword_yake'] = df_keywords_list_unique_pruned['keyword_yake'].progress_apply(
    lambda x: '' if x in stopwords.words('english') else x
)

# Remove empty rows
df_keywords_list_unique_pruned = df_keywords_list_unique_pruned[df_keywords_list_unique_pruned['keyword_yake'] != '']

# Reset index
df_keywords_list_unique_pruned.reset_index(drop=True, inplace=True)

100%|██████████| 69546/69546 [00:03<00:00, 19472.81it/s]


In [15]:
### MAYBE I NEED TO REDO THIS PART WITH THE CLAIM TEXTS INSTEAD OF THE KEYWORDS - NOT CONSIDERED FOR NOW

import spacy

# Download spacy model
nlp = spacy.load('en_core_web_sm')

# Ensure you're working on a new DataFrame, not a slice of an old one
df_keywords_list_unique_pruned_pos = df_keywords_list_unique_pruned.copy()

# Perform part-of-speech tagging on the 'keyword_yake' column 
# and save the POS tags in a new column 'keyword_yake_pos'
df_keywords_list_unique_pruned_pos['keyword_yake_pos'] = df_keywords_list_unique_pruned_pos['keyword_yake'].progress_apply(
    lambda x: [token.pos_ for token in nlp(x)]
)

/home/thiesen/Documents/Cleantech_Concepts/venv/lib64/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
100%|██████████| 69546/69546 [02:46<00:00, 418.77it/s]


In [16]:
# Filter out all keywords that do not contain NOUN, PRON or PROPN in their POS tags
df_keywords_list_unique_pruned_pos = df_keywords_list_unique_pruned_pos[df_keywords_list_unique_pruned_pos['keyword_yake_pos'].progress_apply(
    lambda x: any(pos in ['NOUN', 'PRON', 'PROPN'] for pos in x)
)]

# Reset index
df_keywords_list_unique_pruned_pos.reset_index(drop=True, inplace=True)

100%|██████████| 69546/69546 [00:00<00:00, 918644.12it/s]


In [ ]:
# NOT CONSIDERED FOR NOW
# import re

# # Function to check if a word contains at least three consecutive capital letters (filter out all abbreviations)
# def has_three_consecutive_caps(word):
#     return re.search(r'[A-Z]{3,}', word) is not None

# # Function to check if any word in an n-gram contains at least three consecutive capital letters
# def check_ngram(ngram):
#     return any(has_three_consecutive_caps(word) for word in ngram.split())

# # Use the DataFrame `apply` method to filter rows
# df_keywords_list_unique_pruned_pos_filtered = df_keywords_list_unique_pruned_pos[~df_keywords_list_unique_pruned_pos['keyword_yake'].progress_apply(check_ngram)]

In [ ]:
# NOT CONSIDERED FOR NOW
# # Test to filter out brown corpus words
# from nltk.corpus import brown
# # nltk.download('brown')

# # Step 1: Pre-calculate the set of Brown corpus words
# brown_words_set = set(brown.words())

# # Function to check if any word in a keyword phrase exists in the Brown corpus
# def check_brown_words(phrase):
#     return any(word in brown_words_set for word in phrase.split())

# # Step 2: Use progress_apply with the optimized function
# df_keywords_list_unique_pruned_pos_filtered_brown = df_keywords_list_unique_pruned_pos_filtered[
#     df_keywords_list_unique_pruned_pos_filtered['keyword_yake'].progress_apply(check_brown_words)
# ]

# Match Keyword Dataframe and Patent Ids and CPC Classification

In [18]:
# PatentsView
df_cpc = pd.read_json('/mnt/hdd01/patentsview/Patentsview - Cleantech Patents/df_patentsview_patent_cpc_grouped_cleantech.json')
# Merge df_yake_claims with df_cpc only keep 'cpc' from df_cpc
df_yake_claims_cpc = df_yake_claims.merge(df_cpc[['patent_id', 'cpc']], on='patent_id', how='left')

In [19]:
rows_list = []

# Iterate over rows in dataframe
for index, row in tqdm(df_yake_claims_cpc.iterrows()):
    if row['keywords_yake_exploded'] == []:
        continue
    if not any(keyword for keyword in row['keywords_yake_exploded']):
        continue
    # Iterate over keywords in 'keywords_yake' column and append patent_id and cpc information
    else:
        for keyword in row['keywords_yake_exploded']:
            # PatentsView
            rows_list.append({'patent_id': row['patent_id'], 
                              'cpc': row['cpc'], 
                              'keyword_yake': keyword})

df_cpc_keywords = pd.DataFrame(rows_list)

515742it [00:35, 14416.43it/s]


In [20]:
# Extract 'cpc_subclass' into a new column
df_cpc_keywords['cpc_subclass'] = df_cpc_keywords['cpc'].progress_apply(
    lambda x: [entry['cpc_subclass'] for entry in x.values() if 'cpc_subclass' in entry]
)

# Extract 'cpc_group' into a new column
df_cpc_keywords['cpc_group'] = df_cpc_keywords['cpc'].progress_apply(
    lambda x: [entry['cpc_group'] for entry in x.values() if 'cpc_group' in entry]
)

# Aggregate df_cpc_keywords on 'keyword_yake' column
df_cpc_keywords_agg = df_cpc_keywords.groupby('keyword_yake').agg(
    patent_id_list = pd.NamedAgg(column='patent_id', aggfunc=list),
    cpc_subclass_list = pd.NamedAgg(column='cpc_subclass', aggfunc=list),
    cpc_group_list = pd.NamedAgg(column='cpc_group', aggfunc=list)
)

# Define a function to flatten nested lists
def flatten_nested_list(nested_list):
    return [item for sublist in nested_list for item in sublist]

# Convert nested lists in 'cpc_subclass_list' and 'cpc_group_list' columns to lists of strings
df_cpc_keywords_agg[['cpc_subclass_list', 'cpc_group_list']] = df_cpc_keywords_agg[['cpc_subclass_list', 'cpc_group_list']].applymap(flatten_nested_list)

100%|██████████| 5152903/5152903 [00:07<00:00, 646220.31it/s] 
/tmp/ipykernel_12815/2403604602.py:23: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  df_cpc_keywords_agg[['cpc_subclass_list', 'cpc_group_list']] = df_cpc_keywords_agg[['cpc_subclass_list', 'cpc_group_list']].applymap(flatten_nested_list)


In [21]:
# Remove duplicates from 'cpc_subclass_list'
df_cpc_keywords_agg['cpc_subclass_list'] = df_cpc_keywords_agg['cpc_subclass_list'].progress_apply(lambda x: list(set(x)))

# # Remove duplicates from 'cpc_group_list'
df_cpc_keywords_agg['cpc_group_list'] = df_cpc_keywords_agg['cpc_group_list'].progress_apply(lambda x: list(set(x)))


100%|██████████| 1321218/1321218 [00:07<00:00, 184486.79it/s]


In [23]:
# Merge df_cpc_keywords_agg with df_keywords_list_unique_pruned_pos on 'keyword_yake' column and generate new dataframe
df_cpc_keywords_list = pd.merge(df_keywords_list_unique_pruned_pos, df_cpc_keywords_agg, on='keyword_yake', how='left')

# Keyword Embedding

In [27]:
from sentence_transformers import SentenceTransformer
import torch

In [28]:
model_climatebert = SentenceTransformer('climatebert/distilroberta-base-climate-f')
model_bertforpatents = SentenceTransformer('anferico/bert-for-patents')
model_patentsberta = SentenceTransformer('AI-Growth-Lab/PatentSBERTa')

No sentence-transformers model found with name /home/thiesen/.cache/torch/sentence_transformers/climatebert_distilroberta-base-climate-f. Creating a new one with MEAN pooling.
Some weights of RobertaModel were not initialized from the model checkpoint at /home/thiesen/.cache/torch/sentence_transformers/climatebert_distilroberta-base-climate-f and are newly initialized: ['roberta.pooler.dense.bias', 'roberta.pooler.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.
No sentence-transformers model found with name /home/thiesen/.cache/torch/sentence_transformers/anferico_bert-for-patents. Creating a new one with MEAN pooling.


In [29]:
# Check if GPU is available
if torch.cuda.is_available():
    device = torch.device("cuda")
    print("GPU available: {}".format(torch.cuda.get_device_name(0)))
else:
    device = torch.device("cpu")
    print("GPU not available, CPU used")

GPU available: NVIDIA RTX A4500


In [30]:
# Generate copy of df_cpc_keywords_list
df_keywords_list_unique_embeddings = df_cpc_keywords_list.copy()

# Perform sentence embedding on the 'keyword_yake' (PatentsView) or 'keywords_yake' (EPO) column
df_keywords_list_unique_embeddings['keyword_yake_patentsberta_embedding'] = df_keywords_list_unique_embeddings['keyword_yake'].progress_apply(
    lambda x: model_patentsberta.encode(x)
)

df_keywords_list_unique_embeddings['keyword_yake_climatebert_embedding'] = df_keywords_list_unique_embeddings['keyword_yake'].progress_apply(
    lambda x: model_climatebert.encode(x)
)

df_keywords_list_unique_embeddings['keyword_yake_bertforpatents_embedding'] = df_keywords_list_unique_embeddings['keyword_yake'].progress_apply(
    lambda x: model_bertforpatents.encode(x)
)

  0%|          | 0/61731 [00:00<?, ?it/s]

100%|██████████| 61731/61731 [13:04<00:00, 78.71it/s]


In [31]:
df_keywords_list_unique_embeddings.head()

,keyword_yake,abs_frequency,keyword_yake_pos,patent_id_list,cpc_subclass_list,cpc_group_list,keyword_yake_patentsberta_embedding,keyword_yake_climatebert_embedding,keyword_yake_bertforpatents_embedding
0,forming a source,5,"[VERB, DET, NOUN]","[7026187, 7250333, 8586400, 9431243, 9709335]","[Y02E, Y02P]","[Y02E10/549, Y02E10/547, Y02P90/02, Y02E10/50,...","[-0.2973873, -0.37851676, -0.08203936, -0.0396...","[-0.07330753, -0.020932056, -0.04066288, -0.12...","[-0.25592694, 0.84284246, -0.15500082, 0.47458..."
1,aluminum,615,[NOUN],"[10011900, 10032942, 10046978, 10066308, 10079...","[Y02C, Y02A, Y02W, Y02B, Y02T, Y02E, Y02P]","[Y02B80/10, Y02E10/52, Y02E60/50, Y02W10/37, Y...","[0.06464179, -0.63208365, -0.405623, 0.3495366...","[-0.056519262, 0.03321711, 0.013426696, 0.1743...","[-0.26295653, 0.042311374, -0.09004315, -0.316..."
2,disconnecting the power,7,"[VERB, DET, NOUN]","[5552680, 6081205, 6713989, 7990673, 8547074, ...","[Y02T, Y02E, Y02B, Y02D]","[Y02T90/16, Y02T90/14, Y02D10/00, Y02B20/00, Y...","[-0.23607938, -0.55960715, -0.33855042, -0.061...","[0.046874385, 0.06321991, 0.021172406, 0.05983...","[0.23535673, -0.3955746, -0.069739625, -0.6015..."
3,catalyst regeneration zone,5,"[PROPN, NOUN, NOUN]","[4071436, 4115250, 4115251, 4338788, 5519149]","[Y02T, Y02A, Y02E, Y02P]","[Y02P20/584, Y02E20/14, Y02T50/60, Y02A50/20]","[-0.30876154, -0.31838512, -0.133001, 0.132115...","[0.08325515, -0.014730735, 0.027471442, 0.1283...","[-0.19838068, -0.55406475, -0.80453604, -0.814..."
4,rigidity,21,[NOUN],"[10014352, 10139130, 10164225, 10629794, 10777...","[Y02T, Y02E, Y02A, Y02P]","[Y02T10/62, Y02E10/72, Y02E10/549, Y02E60/50, ...","[0.12807769, -0.8062214, -0.28694445, 0.236054...","[0.0015207138, -0.012041591, 0.032385968, 0.00...","[-0.12907217, 0.72338533, -1.0261452, 0.242813..."


In [32]:
# Save dataframe to json - Make sure to use correct directory (PatentsView vs. EPO)
df_keywords_list_unique_embeddings.to_json('/mnt/hdd01/patentsview/Patentsview - Cleantech Patents/Cleantech Concepts/Yake/g_patent_claims_cleantech_yake_keywords_embeddings.json', orient='records')

In [34]:
df_keywords_list_unique_embeddings.to_json('/mnt/hdd01/patentsview/Similarity Search - CPC Classification and Claims/df_keywords_y02_uspto_embeddings_processed.json', orient='records')

# Dimensionality Reduction

In [ ]:
# Load data if necessary
df_keywords_list_unique_embeddings = pd.read_json('/mnt/hdd01/patentsview/Patentsview - Cleantech Patents/Cleantech Concepts/Yake/g_patent_claims_cleantech_yake_keywords_embeddings.json')

## UMAP

In [ ]:
import umap
import numpy as np
umap_dim = 2
reducer = umap.UMAP(random_state=42, n_components=umap_dim)

In [ ]:
# Stack all the embeddings into a 2D array
embeddings_patentsberta = np.vstack(df_keywords_list_unique_embeddings['keyword_yake_patentsberta_embedding'].values)

# Perform UMAP dimensionality reduction
umap_embeddings_patentsberta = reducer.fit_transform(embeddings_patentsberta)

# Assign the reduced dimension embeddings back to new DataFrame columns
df_keywords_list_unique_embeddings_umap['keyword_yake_patentsberta_embedding_umap'] = list(umap_embeddings_patentsberta)

## t-SNE

In [ ]:
import numpy as np
from openTSNE import TSNE
from sklearn.model_selection import train_test_split

In [ ]:
# Generate train/test split
patentsberta_x_train, patentsberta_x_test = train_test_split(df_keywords_list_unique_embeddings['keyword_yake_patentsberta_embedding'].tolist(), test_size=0.2, random_state=42)
patentsberta_x_train_np = np.array(patentsberta_x_train)
patentsberta_x_test_np = np.array(patentsberta_x_test)

In [ ]:
tsne = TSNE(
    perplexity=30,
    metric="euclidean",
    n_jobs=8,
    random_state=42,
    n_iter=1000,
    verbose=True,
)

In [ ]:
# Perform t-SNE on training data
patentsberta_embedding_train = tsne.fit(patentsberta_x_train_np)

In [ ]:
patentsberta_embedding_test = patentsberta_embedding_train.transform(patentsberta_x_test_np)

In [ ]:
df_keywords_list_unique_embeddings_tsne = df_keywords_list_unique_embeddings.copy()

patentsberta_embedding = np.concatenate((patentsberta_embedding_train, patentsberta_embedding_test), axis=0)
df_keywords_list_unique_embeddings_tsne['keyword_yake_patentsberta_embedding_tsne'] = patentsberta_embedding.tolist()
df_keywords_list_unique_embeddings_tsne['keyword_yake_patentsberta_embedding_tsne_x'] = df_keywords_list_unique_embeddings_tsne['keyword_yake_patentsberta_embedding_tsne'].apply(lambda x: x[0])
df_keywords_list_unique_embeddings_tsne['keyword_yake_patentsberta_embedding_tsne_y'] = df_keywords_list_unique_embeddings_tsne['keyword_yake_patentsberta_embedding_tsne'].apply(lambda x: x[1])

# Clustering

In [ ]:
# Import HDBSCAN
import hdbscan

In [ ]:
# Ensure you're working on a new DataFrame, not a slice of an old one
df_keywords_list_unique_embeddings_hdbscan = df_keywords_list_unique_embeddings_tsne.copy()

# Perform HDBSCAN clustering on the UMAP coordinates
clusterer_patentsberta = hdbscan.HDBSCAN(min_cluster_size=100, min_samples=1).fit(df_keywords_list_unique_embeddings_hdbscan[['keyword_yake_patentsberta_embedding_tsne_x', 'keyword_yake_patentsberta_embedding_tsne_y']])

# Assign the cluster labels back to the DataFrame
df_keywords_list_unique_embeddings_hdbscan['patentsberta_cluster'] = clusterer_patentsberta.labels_

# Erase all rows with cluster -1
df_keywords_list_unique_embeddings_hdbscan = df_keywords_list_unique_embeddings_hdbscan[df_keywords_list_unique_embeddings_hdbscan['patentsberta_cluster'] != -1]

In [ ]:
df_keywords_list_unique_embeddings_hdbscan.to_json('/mnt/hdd01/patentsview/Patentsview - Cleantech Patents/Cleantech Concepts/Yake/g_patent_claims_cleantech_yake_keywords_embeddings_tsne_hdbscan.json', orient='records')

# Plotting

In [ ]:
import plotly.express as px

In [ ]:
# Plot the UMAP embeddings in three figures, color coded by cluster
fig_patentsberta = px.scatter(
    df_keywords_list_unique_embeddings_hdbscan, 
    x='keyword_yake_patentsberta_embedding_tsne_x', 
    y='keyword_yake_patentsberta_embedding_tsne_y', 
    color='patentsberta_cluster', 
    hover_data=['patentsberta_cluster', 'keyword_yake'], 
    title='Cleantech Yake Keywords - HDBSCAN, TSNE, PatentSBERTa',
    height=800,  # Adjust as needed
    width=800    # Adjust as needed
)
# Display the figures
fig_patentsberta.show()

# fig_patentsberta.write_html('/mnt/hdd01/patentsview/Patentsview - Cleantech Patents/Cleantech Concepts/Yake/g_patent_claims_cleantech_yake_keywords_embeddings_tsne.html')

# Name Cluster

In [ ]:
from transformers import pipeline, set_seed

In [ ]:
# Build dataframe of keywords and their corresponding clusters, where each row is a cluster with list of keywords
df_keywords_clusters_patentsberta = df_keywords_list_unique_embeddings_hdbscan.groupby('patentsberta_cluster')['keyword_yake'].apply(list).reset_index(name='keywords')

In [ ]:
# Initialize a text-generation pipeline with Flan-T5-large
import torch
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
generator = pipeline('text2text-generation', model='google/flan-t5-large', device=device)
set_seed(42)

In [ ]:
def generate_cluster_name(keywords):
    # Ensure the keywords are in a list format
    keywords = keywords.split(', ') if isinstance(keywords, str) else keywords
    # Select only the first 1500 keywords from the list
    selected_keywords = keywords[:2000]
    # Join the selected keywords into a string format
    keywords_str = ', '.join(selected_keywords)
    # Create a prompt from the selected keywords
    # prompt = f"Based on the following keywords, come up with a specific, precise and short topic name: {keywords_str}"
    prompt = f"Generate a concise and descriptive common theme or category for a cluster containing the following keywords: {keywords_str}." # The name should be in title case and should not exceed three words."
    # Doesn't work at all -> only focuses on electric vehicle innovation# prompt = f"Given the keywords: {keywords_str}, provide a succinct cluster name similar to how 'Electric Vehicle Innovation' represents keywords like 'battery technology, electric motor, charging infrastructure'."
    # prompt = f"Identify a common theme or category for the following keywords: {keywords_str}. Provide a concise, descriptive name for this theme or category."
    # prompt = f"The keywords {keywords_str} all belong to the category: _____"
    # Generate a response using the GPT-3 model
    response = generator(prompt, max_length=10, do_sample=True, temperature=0.8)[0]['generated_text']
    # Extract the cluster name from the response
    cluster_name = response
    return cluster_name

# Apply the function to the 'keywords' column to generate cluster names
df_keywords_clusters_patentsberta['cluster_name'] = df_keywords_clusters_patentsberta['keywords'].progress_apply(generate_cluster_name)

In [ ]:
# Merge df_keywords_clusters_patentsberta with df_keywords_list_unique_pruned_embeddings_hdbscan on 'patentsberta_cluster'
df_keywords_list_unique_embeddings_cluster = pd.merge(df_keywords_list_unique_embeddings_hdbscan, df_keywords_clusters_patentsberta[['patentsberta_cluster', 'cluster_name']], on='patentsberta_cluster', how='left')

In [ ]:
# Plotting
fig_patentsberta_cluster = px.scatter(
    df_keywords_list_unique_embeddings_cluster, 
    x='keyword_yake_patentsberta_embedding_tsne_x', 
    y='keyword_yake_patentsberta_embedding_tsne_y', 
    color='cluster_name', 
    hover_data=['patentsberta_cluster', 'keyword_yake'], 
    title='Cleantech Yake Keywords - HDBSCAN, TSNE, PatentSBERTa',
    height=800,  # Adjust as needed
    width=1200    # Adjust as needed
)

# Display the figures
fig_patentsberta_cluster.show()

In [ ]:
# Arrange column order
df_keywords_clusters_patentsberta = df_keywords_clusters_patentsberta[['patentsberta_cluster', 'cluster_name', 'keywords']]

In [ ]:
df_keywords_clusters_patentsberta.to_json('/mnt/hdd01/patentsview/Patentsview - Cleantech Patents/Cleantech Concepts/Yake/g_patent_claims_cleantech_yake_keywords_clusters_patentsberta.json', orient='records')

In [ ]:
df_keywords_list_unique_embeddings_cluster.to_json('/mnt/hdd01/patentsview/Patentsview - Cleantech Patents/Cleantech Concepts/Yake/g_patent_claims_cleantech_yake_keywords_embeddings_tsne_hdbscan_cluster.json', orient='records')